# Indexing and selecting data

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    import seaborn
except ImportError:
    pass

In [2]:
# redefining the example objects

# series
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

# dataframe
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,country,population,area,capital
0,Belgium,11.3,30510,Brussels
1,France,64.3,671308,Paris
2,Germany,81.3,357050,Berlin
3,Netherlands,16.9,41526,Amsterdam
4,United Kingdom,64.9,244820,London


Setting the index to the country names:

In [3]:
countries = countries.set_index('country')
countries

,population,area,capital
country,,,
Belgium,11.3,30510,Brussels
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
Netherlands,16.9,41526,Amsterdam
United Kingdom,64.9,244820,London


## Some notes on selecting data

One of pandas' basic features is the labeling of rows and columns, but this makes indexing also a bit more complex compared to numpy. We now have to distuinguish between:

- selection by label
- selection by position.

### `data[]` provides some convenience shortcuts 

For a DataFrame, basic indexing selects the columns.

Selecting a single column:

In [4]:
countries['area']

country
Belgium            30510
France            671308
Germany           357050
Netherlands        41526
United Kingdom    244820
Name: area, dtype: int64

or multiple columns:

In [5]:
countries[['area', 'population']]

,area,population
country,,
Belgium,30510,11.3
France,671308,64.3
Germany,357050,81.3
Netherlands,41526,16.9
United Kingdom,244820,64.9


But, slicing accesses the rows:

In [6]:
countries['France':'Netherlands']

,population,area,capital
country,,,
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
Netherlands,16.9,41526,Amsterdam


<div class="alert alert-danger">
    <b>NOTE</b>: Unlike slicing in numpy, the end label is **included**.
</div>

So as a summary, `[]` provides the following convenience shortcuts:

- Series: selecting a label: `s[label]`
- DataFrame: selecting a single or multiple columns: `df['col']` or `df[['col1', 'col2']]`
- DataFrame: slicing the rows: `df['row_label1':'row_label2']` or `df[mask]`

### Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

These methods index the different dimensions of the frame:

* `df.loc[row_indexer, column_indexer]`
* `df.iloc[row_indexer, column_indexer]`

Selecting a single element:

In [7]:
countries.loc['Germany', 'area']

357050

But the row or column indexer can also be a list, slice, boolean array, ..

In [8]:
countries.loc['France':'Germany', ['area', 'population']]

,area,population
country,,
France,671308,64.3
Germany,357050,81.3


---
Selecting by position with `iloc` works similar as indexing numpy arrays:

In [9]:
countries.iloc[0:2,1:3]

,area,capital
country,,
Belgium,30510,Brussels
France,671308,Paris


The different indexing methods can also be used to assign data:

In [10]:
countries2 = countries.copy()
countries2.loc['Belgium':'Germany', 'population'] = 10

In [11]:
countries2

,population,area,capital
country,,,
Belgium,10.0,30510,Brussels
France,10.0,671308,Paris
Germany,10.0,357050,Berlin
Netherlands,16.9,41526,Amsterdam
United Kingdom,64.9,244820,London


## Boolean indexing (filtering)

Often, you want to select rows based on a certain condition. This can be done with 'boolean indexing' (like a where clause in SQL). 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [12]:
countries['area'] > 100000

country
Belgium           False
France             True
Germany            True
Netherlands       False
United Kingdom     True
Name: area, dtype: bool

In [13]:
countries[countries['area'] > 100000]

,population,area,capital
country,,,
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
United Kingdom,64.9,244820,London


---

<div class="alert alert-success">
    <b>EXERCISE</b>: Add a column `density` with the population density (note: population column is expressed in millions)
</div>

In [14]:
countries["density"] = countries.population / countries.area * 1000000
countries

,population,area,capital,density
country,,,,
Belgium,11.3,30510,Brussels,370.370370
France,64.3,671308,Paris,95.783158
Germany,81.3,357050,Berlin,227.699202
Netherlands,16.9,41526,Amsterdam,406.973944
United Kingdom,64.9,244820,London,265.092721


<div class="alert alert-success">
    <b>EXERCISE</b>: Select the capital and the population column of those countries where the density is larger than 300
</div>

In [15]:
dense = countries.density > 300
countries[["population", "capital"]][dense]

,population,capital
country,,
Belgium,11.3,Brussels
Netherlands,16.9,Amsterdam


<div class="alert alert-success">
    <b>EXERCISE</b>: Add a column 'density_ratio' with the ratio of the density to the mean density
</div>

In [16]:
countries["density_ratio"] = countries.density / countries.density.mean()
countries

,population,area,capital,density,density_ratio
country,,,,,
Belgium,11.3,30510,Brussels,370.370370,1.355755
France,64.3,671308,Paris,95.783158,0.350618
Germany,81.3,357050,Berlin,227.699202,0.833502
Netherlands,16.9,41526,Amsterdam,406.973944,1.489744
United Kingdom,64.9,244820,London,265.092721,0.970382


<div class="alert alert-success">
    <b>EXERCISE</b>: Change the capital of the UK to Cambridge
</div>

In [17]:
UK = countries.loc["United Kingdom"]
UK.capital = "Cambridge"
UK.capital

C:\Users\usppa\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


'Cambridge'

<div class="alert alert-success">
    <b>EXERCISE</b>: Select all countries whose population density is between 100 and 300 people/km²
</div>

In [18]:
countries[(countries.density > 100) & (countries.density < 300)]

,population,area,capital,density,density_ratio
country,,,,,
Germany,81.3,357050,Berlin,227.699202,0.833502
United Kingdom,64.9,244820,London,265.092721,0.970382


## Some other useful methods: `isin` and string methods

The `isin` method of Series is very useful to select rows that may contain certain values:

In [19]:
s = countries['capital']

In [20]:
s.isin?

In [21]:
s.isin(['Berlin', 'London'])

country
Belgium           False
France            False
Germany            True
Netherlands       False
United Kingdom     True
Name: capital, dtype: bool

This can then be used to filter the dataframe with boolean indexing:

In [22]:
countries[countries['capital'].isin(['Berlin', 'London'])]

,population,area,capital,density,density_ratio
country,,,,,
Germany,81.3,357050,Berlin,227.699202,0.833502
United Kingdom,64.9,244820,London,265.092721,0.970382


Let's say we want to select all data for which the capital starts with a 'B'. In Python, when having a string, we could use the `startswith` method:

In [23]:
'Berlin'.startswith('B')

True

In pandas, these are available on a Series through the `str` namespace:

In [24]:
countries['capital'].str.startswith('B')

country
Belgium            True
France            False
Germany            True
Netherlands       False
United Kingdom    False
Name: capital, dtype: bool

For an overview of all string methods, see: http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling

<div class="alert alert-success">
    <b>EXERCISE</b>: Select all countries that have capital names with more than 7 characters
</div>

In [25]:
long_name = countries.capital.str.len() >7
countries[long_name]

,population,area,capital,density,density_ratio
country,,,,,
Belgium,11.3,30510,Brussels,370.370370,1.355755
Netherlands,16.9,41526,Amsterdam,406.973944,1.489744


<div class="alert alert-success">
    <b>EXERCISE</b>: Select all countries that have capital names that contain the character sequence 'am'
</div>

## Pitfall: chained indexing (and the 'SettingWithCopyWarning')

In [26]:
countries.loc['Belgium', 'capital'] = 'Ghent' 

In [27]:
countries

,population,area,capital,density,density_ratio
country,,,,,
Belgium,11.3,30510,Ghent,370.370370,1.355755
France,64.3,671308,Paris,95.783158,0.350618
Germany,81.3,357050,Berlin,227.699202,0.833502
Netherlands,16.9,41526,Amsterdam,406.973944,1.489744
United Kingdom,64.9,244820,London,265.092721,0.970382


In [28]:
countries['capital']['Belgium'] = 'Antwerp' 

<ipython-input-28-f99e101c5201>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries['capital']['Belgium'] = 'Antwerp'


In [29]:
countries

,population,area,capital,density,density_ratio
country,,,,,
Belgium,11.3,30510,Antwerp,370.370370,1.355755
France,64.3,671308,Paris,95.783158,0.350618
Germany,81.3,357050,Berlin,227.699202,0.833502
Netherlands,16.9,41526,Amsterdam,406.973944,1.489744
United Kingdom,64.9,244820,London,265.092721,0.970382


In [30]:
countries[countries['capital'] == 'Antwerp']['capital'] = 'Brussels' 

<ipython-input-30-50ff5c336503>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries[countries['capital'] == 'Antwerp']['capital'] = 'Brussels'


In [31]:
countries

,population,area,capital,density,density_ratio
country,,,,,
Belgium,11.3,30510,Antwerp,370.370370,1.355755
France,64.3,671308,Paris,95.783158,0.350618
Germany,81.3,357050,Berlin,227.699202,0.833502
Netherlands,16.9,41526,Amsterdam,406.973944,1.489744
United Kingdom,64.9,244820,London,265.092721,0.970382


How to avoid this?

* Use `loc` instead of chained indexing if possible!
* Or `copy` explicitly if you don't want to change the original data.

## More exercises!

For the quick ones among you, here are some more exercises with some larger dataframe with film data. These exercises are based on the [PyCon tutorial of Brandon Rhodes](https://github.com/brandon-rhodes/pycon-pandas-tutorial/) (so all credit to him!) and the datasets he prepared for that. You can download these data from here: [`titles.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKajNMa1pfSzN6Q3M) and [`cast.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKal9UYTJSR2ZhSW8) and put them in the `/data` folder.

In [32]:
cast = pd.read_csv('data/cast.csv')
cast.head()

,title,year,name,type,character,n
0,Suuri illusioni,1985,Homo $,actor,Guests,22.0
1,Gangsta Rap: The Glockumentary,2007,Too $hort,actor,Himself,NaN
2,Menace II Society,1993,Too $hort,actor,Lew-Loc,27.0
3,Porndogs: The Adventures of Sadie,2009,Too $hort,actor,Bosco,3.0
4,Stop Pepper Palmer,2014,Too $hort,actor,Himself,NaN


In [33]:
titles = pd.read_csv('data/titles.csv')
titles.head()

,title,year
0,The Rising Son,1990
1,Ashes of Kukulcan,2016
2,The Thousand Plane Raid,1969
3,Crucea de piatra,1993
4,The 86,2015


<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies are listed in the titles dataframe?
</div>

In [35]:
len(titles)

215981

<div class="alert alert-success">
    <b>EXERCISE</b>: What are the earliest two films listed in the titles dataframe?
</div>

In [37]:
titles.year.sort_values()

165182    1894
85708     1897
115903    1899
198354    1899
12582     1900
          ... 
26537     2022
185368    2022
40476     2022
199790    2024
44198     2024
Name: year, Length: 215981, dtype: int64

In [42]:
titles.loc[165182]

title    Miss Jerry
year           1894
Name: 165182, dtype: object

In [43]:
titles.loc[85708]

title    Reproduction of the Corbett and Fitzsimmons Fight
year                                                  1897
Name: 85708, dtype: object

<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies have the title "Hamlet"?
</div>

In [52]:
boolean = titles.title == "Hamlet"
boolean[boolean].index.size

19

<div class="alert alert-success">
    <b>EXERCISE</b>: List all of the "Treasure Island" movies from earliest to most recent.
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies were made from 1950 through 1959?
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: How many roles in the movie "Inception" are NOT ranked by an "n" value?
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: But how many roles in the movie "Inception" did receive an "n" value?
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: Display the cast of "North by Northwest" in their correct "n"-value order, ignoring roles that did not earn a numeric "n" value.
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: How many roles were credited in the silent 1921 version of Hamlet?
</div>

<div class="alert alert-success">
    <b>EXERCISE</b>: List the supporting roles (having n=2) played by Cary Grant in the 1940s, in order by year.
</div>